In [8]:
import sys
import urllib.request
import socket
import socks
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import concurrent.futures
import requests
import warnings
import random
warnings.filterwarnings("ignore")
from requests.structures import CaseInsensitiveDict
#df = pd.read_csv('./est_price_final.csv')

In [9]:
df = pd.read_csv('./sample_data_2k_res_1.csv')

In [10]:
df['property_estimate'] = '-'
df['rental_estimate'] = '-'
df['beds_zoopla'] = '-'

In [11]:
def get_my_version(new_df,index):
    row = new_df.iloc[index]
    cur_id = str(row['id'])
    headers = CaseInsensitiveDict()
    headers["user-agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"
    url = 'https://www.zoopla.co.uk/property/estimated-price/'+cur_id
    try:
        req = requests.get(url,headers=headers)
        if req.status_code != 200:
            return ('unavailable','unavailable','unavailable',index)
    except:
        return ('unavailable','unavailable','unavailable',index)
    html = bs(req.text,'html.parser')
    #print(html)
    price1 = 'unavailable'
    price2 = 'unavailable'
    bedroom = 'unavailable'
    try:
        price1 = html.find('p',text = re.compile('Estimated price')).parent.find_all('p')[-1].text
    except:
        pass
    try:
        s = str(req.text)
        st_ps = s.find('currentPrice')
        st_ps = s.find('currentPrice',st_ps+2)
        end_ps = st_ps
        while(s[end_ps]!=','):
            end_ps+=1
        price2 = s[st_ps+14:end_ps]
    except Exception as e:
        #print(e)
        pass

    try:
        bedroom = html.find('p',class_="css-z5sht9-Text-StyledText et73qvw1").text
    except:
        pass

    return (price1,price2,bedroom,index)

In [12]:
get_my_version(df,535)

('£329,000', '1200', '4beds', 535)

In [13]:
tot_fnd = 0
cur = 0
import timeit
start = timeit.default_timer()
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(get_my_version,new_df,x): x for x in range(0,100)}
    for future in concurrent.futures.as_completed(future_to_url):
        res = future.result()
        
        df.loc[res[3],'property_estimate'] = res[0]
        df.loc[res[3],'rental_estimate'] = res[1]
        df.loc[res[3],'beds_zoopla'] = res[2]
        
        if res[0]!='unavailable':
            tot_fnd+=1
        cur+=1
        
        if cur % 10 == 0:
            print(cur,tot_fnd,'time to save . . .')
            
stop = timeit.default_timer()
print('Time: ', stop - start)  

10 6 time to save . . .
20 15 time to save . . .
30 25 time to save . . .
40 35 time to save . . .
50 45 time to save . . .
60 54 time to save . . .
70 63 time to save . . .
80 73 time to save . . .
90 81 time to save . . .
100 91 time to save . . .
Time:  13.78357619999997


In [14]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,level_0,index,date,address,status,beds,type,...,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,TOTAL_FLOOR_AREA,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,NUMBER_HABITABLE_ROOMS,id,property_estimate,rental_estimate,beds_zoopla
0,0,0,0,12,2191039,13.04.2021,"Berrichon Crescent, Whitehouse, Milton Keynes,...",Added,5,Detached House For Sale,...,86,92,196.0,74,45.0,NaN,32455762,"£647,000",2450,5beds
1,1,1,1,40,1418831,04.06.2020,"Primrose Walk, Warminster",Added,3,Semi-Detached House For Sale,...,69,86,68.0,222,97.0,5.0,1199004,"£235,000",750,3beds
2,2,2,2,118,2246204,08.03.2021,"Lamberhurst House, 13 Lovelinch Close, London,...",Reduced,3,Maisonette For Sale,...,75,81,74.57,207,153.0,3.0,21055134,"£348,000",1400,3beds
3,3,3,3,155,985219,20.03.2020,"Main Street, Ingleby, Derby",Added,4,Equestrian Facility For Sale,...,13,90,173.0,866,170.0,6.0,6627597,"£524,000",2000,4beds
4,4,4,4,190,1693903,13.04.2021,"Queen Street, Coggeshall",Added,2,Terraced House For Sale,...,71,87,63.0,187,63.0,3.0,5077659,"£332,000",1150,2beds
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,1995,0,38,1358261,08.12.2020,"Wimblestone Road, WINSCOMBE",Added,3,Semi-Detached House For Sale,...,63,84,111.0,234,84.0,6.0,2867184,-,-,-
1996,1996,1996,1,233,1135805,21.03.2020,"Church Street, Broadstairs",Added,2,Terraced House For Sale,...,85,97,66.0,79,-15.0,NaN,32668837,-,-,-
1997,1997,1997,2,436,2034719,11.06.2021,"The Hawnelands,Halesowen,B63 3RT",Added,3,Semi-Detached House For Sale,...,58,75,127.0,250,144.0,6.0,803799,-,-,-
1998,1998,1998,3,587,1648853,05.03.2021,"Swale Avenue, Peterborough",Added,3,Semi-Detached House For Sale,...,67,82,83.0,218,109.0,4.0,18003380,-,-,-
